# Bronze Layer - Data Collection from PT Evaluations

In [4]:
import numpy as np
import pandas as pd
import requests
from azure.storage.blob import BlobServiceClient
from datetime import datetime


connection_string = "DefaultEndpointsProtocol=https;AccountName=ptevalstorage;AccountKey=5U/PV3+pn2bBU8RGIQ26QRg5du1C1KkzV28gOfCYKM/XrzJwu23F1nhqkOXhHsJkbq9Be+RzkOLg+AStb0TPQQ==;EndpointSuffix=core.windows.net"
container_name = "ptevals"

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)
blobs_to_load = []
eval_date = []

for blob in container_client.list_blobs():
    if blob.creation_time and blob.creation_time.strftime("%Y-%m-%d") >= last_added_date:
        blobs_to_load.append(blob.name)
        eval_date.append(blob.creation_time.strftime("%Y-%m-%d"))



# instream of blob text
def read_blob_to_string(blob_name: str) -> str:
    blob_url = f"https://{blob_service_client.account_name}.blob.core.windows.net/{container_name}/{blob_name}"
    response = requests.get(blob_url)
    return response.text


StatementMeta(, 13ab6e47-def1-4253-9e4e-d67e1a4d87c6, 8, Finished, Available, Finished)

In [5]:
# Feature information

age_sex = []
fusion = []
ordering_provider = []
brace = []
pain = []
plof = []
mobility = []

# Process each blob
for blob in blobs_to_load:
    
    # Read the blob content
    content = read_blob_to_string(blob)
    lines = content.split('\n')    
    collect_plof = False
    collect_mobility = False
    curr_mobility = []
    curr_plof = []
    
    ## Initial data collection from evaluations
    for line in lines:
        line = line.lower()
        if line.startswith('reason for admission:'):
            fusion.append(line.split('reason for admission:')[1].strip())
        if line.startswith('ordered by:'):
            ordering_provider.append(line.split('ordered by:')[1].strip())
        if line.startswith('precautions:'):
            brace.append(line.split('precautions:')[1].strip())
        if line.startswith('subjective:'):
            pain.append(line.split('subjective:')[1].strip())
        if line.startswith('living situation/prior level of function:'):
            collect_plof = True
            curr_plof.append(line.split('living situation/prior level of function:')[1].strip())
            continue
        if line.startswith("objective:"):
            collect_plof = False
        if collect_plof:
            curr_plof.append(line)
        if line.startswith('appearance:'):
            age_sex.append(line.split('appearance:')[1].strip())
        if line.startswith('mobility assessment:'):
            collect_mobility = True
            continue
        if line.startswith("assessment:"):
            collect_mobility = False
        if collect_mobility:
            curr_mobility.append(line)
    mobility.append(''.join(curr_mobility))
    plof.append(''.join(curr_plof))

StatementMeta(, 13ab6e47-def1-4253-9e4e-d67e1a4d87c6, 9, Finished, Available, Finished)

In [7]:
# DataFrame assembly
columns = ['eval_date', 'age_sex', 'fusion', 'ordering_provider', 'brace', 'pain', 'plof', 'mobility']
data = list(zip(eval_date, age_sex, fusion, ordering_provider, brace, pain, plof, mobility))
df = spark.createDataFrame(data, schema = columns)

df.write.mode('append').saveAsTable('bronze_layer_nlp')

StatementMeta(, 13ab6e47-def1-4253-9e4e-d67e1a4d87c6, 11, Finished, Available, Finished)